## 程序框架


**程序目录**

- include/
- src/
- main.cc
- CMakeLists.txt
- vis.mac
- init_vis.mac


## 文件 CMakeLists.txt

*CMakeLists.txt* 原则上不需要改动，如果需要将主目录下的文件复制到编译的文件夹，则参考以下代码进行修改。

```cmake
# 以下代码将 init.mac、init_vis.mac、vis.mac、wu.in、wu.out 这5个文件复制到
set(COPY_SCRIPTS
   init.mac
   init_vis.mac
   vis.mac
   wu.in
   wu.out
  )

foreach(_script ${COPY_SCRIPTS})
  configure_file(
    ${PROJECT_SOURCE_DIR}/${_script}
    ${PROJECT_BINARY_DIR}/${_script}
    COPYONLY
    )
endforeach()
```

## 文件 main.cc

```cpp
int main(int argc,char** argv)
{
  // 随机数设置 如果需要的话
  // G4Random::setTheEngine(new CLHEP::RanecuEngine);
  // G4int seconds =  time(NULL);
  // G4Random::setTheSeed(seconds);


  // Construct the default run manager
  G4MTRunManager* mtrunManager = NULL;

  mtrunManager = new G4MTRunManager;
  mtrunManager->SetNumberOfThreads(2);
  // mtrunManager->SetUserInitialization(new wuWorkerInitialization);


  // Set mandatory initialization classes ，410版本的框架是这样的，ActionInitialization来管理。
  //
  // Detector construction
  mtrunManager->SetUserInitialization(new wuDetectorConstruction());


  // Physics list
  // G4VModularPhysicsList* physicsList = new wuPhysicsList();
  mtrunManager->SetUserInitialization(new FTFP_BERT_HP());
  
  // User action initialization
  mtrunManager->SetUserInitialization(new wuActionInitialization());


  // Initialize G4 kernel
  mtrunManager->Initialize();


    
  G4VisManager* visManager = 0;
  G4UImanager* UImanager = G4UImanager::GetUIpointer();

  G4String commandopt = argv[1];
  if(commandopt == "vis.mac")// 开启图形界面模式
    {
      visManager = new G4VisExecutive;
      visManager->Initialize();

      G4UIExecutive* ui = new G4UIExecutive(argc, argv);
      UImanager->ApplyCommand("/control/execute vis.mac");
      ui->SessionStart();
      delete ui;
    }
  else
    {
      if(commandopt == "-l")// 开启命令行模式
	{  
	  G4UIsession* session = new G4UIterminal(new G4UItcsh);
	  session->SessionStart();
	  delete session;
	}
      else
	{// 无图形界面执行脚本模式
	  G4String command = "/control/execute ";
	  G4String fileName = argv[1];
	  UImanager->ApplyCommand(command + fileName);
	}
    }

  
  // Job termination
  // Free the store: user actions, physics_list and detector_description are
  // owned and deleted by the run manager, so they should not be deleted 
  // in the main() program !
  
  if(visManager != 0)
    delete visManager;

  delete mtrunManager;

  return 0;
}
```



G4RunManager 管理运行过程。G4MTRunManager 是 G4RunManager 在多线程模式下的替代。在执行函数 Initialize() 之后，才创建并启用线程。

在函数 Initialize()  之前，用户定义的所有 user initialization 类都应该赋值给 G4RunManager/G4MTRunManager。这些用户类的赋值由 SetuserInitialization() 函数完成。



G4RunManager/G4MTRunManager 常用的函数: 

```cpp
Initialize()
```

- 探测器几何结构的构建和灵敏探测器和/或数字化模块的设置
- 粒子和物理过程的构建
- 截面表的计算


```cpp
BeamOn(G4int numberOfEvent)
```

设置模拟的事件数，并启动模拟。



### Physics list

**Physics list** 对于入门级别来说，选择以下哪个区别不大

```cpp
#include "wuPhysicsList.hh"
#include "FTF_BIC.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysic G4HadronPhysicsFTF_BIC G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut
#include "FTFP_BERT_ATL.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysic G4HadronPhysicsFTFP_BERT_ATL G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut 
#include "FTFP_BERT.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysic G4HadronPhysicsFTFP_BERT G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut        
#include "FTFP_BERT_HP.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysicsHP G4HadronPhysicsFTFP_BERT_HP G4StoppingPhysics G4IonPhysics            
#include "FTFP_BERT_TRV.hh"// G4EmStandardPhysicsGS G4EmExtraPhysics G4DecayPhysics G4HadronHElasticPhysics G4HadronPhysicsFTFP_BERT_TRV G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut
#include "FTFP_INCLXX.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysics G4HadronPhysicsINCLXX G4StoppingPhysics G4IonINCLXXPhysics G4NeutronTrackingCut
#include "FTFP_INCLXX_HP.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysicsHP G4HadronPhysicsINCLXX G4StoppingPhysics G4IonINCLXXPhysics
#include "G4GenericPhysicsList.hh"                    
#include "LBE.hh"// 这个比较复杂 -_-
#include "NuBeam.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysics G4HadronPhysicsNuBeam G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut
#include "QBBC.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysicsXS G4StoppingPhysics G4IonPhysics G4HadronInelasticQBBC G4NeutronTrackingCut
#include "QGS_BIC.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysics G4HadronPhysicsQGS_BIC G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut
#include "QGSP_BERT.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysics G4HadronPhysicsQGSP_BERT G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut
#include "QGSP_BERT_HP.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysicsHP G4HadronPhysicsQGSP_BERT_HP G4StoppingPhysics G4IonPhysics
#include "QGSP_BIC_AllHP.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysicsPHP G4HadronPhysicsQGSP_BIC_AllHP G4StoppingPhysics G4IonPhysicsPHP
#include "QGSP_BIC.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysics G4HadronPhysicsQGSP_BIC G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut
#include "QGSP_BIC_HP.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysicsHP G4HadronPhysicsQGSP_BIC_HP G4StoppingPhysics G4IonPhysics
#include "QGSP_FTFP_BERT.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysics G4HadronPhysicsQGSP_FTFP_BERT G4StoppingPhysics G4IonPhysics G4NeutronTrackingCut
#include "QGSP_INCLXX.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysics G4HadronPhysicsINCLXX G4StoppingPhysics G4IonINCLXXPhysics G4NeutronTrackingCut
#include "QGSP_INCLXX_HP.hh"// G4EmStandardPhysics G4EmExtraPhysics G4DecayPhysics G4HadronElasticPhysicsHP G4HadronPhysicsINCLXX G4StoppingPhysics G4IonINCLXXPhysics
#include "Shielding.hh"// 这个比较复杂,分好几种情况 -_-
```


## 文件 wuActionInitialization.cc

```cpp
void wuActionInitialization::Build() const
{
  //源的设置
  SetUserAction(new wuPrimaryGeneratorActionAll);

  
#if G4VERSION_NUMBER >= 1030
  
  G4MultiRunAction* actsRun = new G4MultiRunAction;
  G4MultiEventAction* actsEvent = new G4MultiEventAction;
  G4MultiTrackingAction* actsTrack = new G4MultiTrackingAction;
  G4MultiSteppingAction* actsStep = new G4MultiSteppingAction;
  
  //...0123456789876543210...0123456789876543210...

  actsRun->push_back(G4UserRunActionUPtr(new wuRunActionAll));
  actsEvent->push_back(G4UserEventActionUPtr(new wuEventActionAll));
  actsTrack->push_back(G4UserTrackingActionUPtr(new wuTrackingActionAll));
  actsStep->push_back(G4UserSteppingActionUPtr(new wuSteppingActionAll));
  
  SetUserAction(new wuStackingActionAll);


  //...0123456789876543210...0123456789876543210...
  
  SetUserAction(actsRun);
  SetUserAction(actsTrack);
  SetUserAction(actsEvent);// 输出 SD 的数据，不输出则注释本行
  SetUserAction(actsStep);// 输出 step 的数据，不输出则注释本行
  
#else
  G4cout<<"It need G4VERSION_NUMBER >= 1030"<<G4endl;
#endif 
}  
```


In [1]:
!jupyter nbconvert MAIN.ipynb --to html

[NbConvertApp] Converting notebook MAIN.ipynb to html

[NbConvertApp] Writing 584246 bytes to MAIN.html

